### Lấy dữ liệu từ [SoundCloud](soundcloud.com) API

19127216 - Đặng Hoàn Mỹ

Để tiết kiệm thời gian lấy data, sử dụng concurrent để request đến nhiều link cùng một lúc `max_workers=25`.

[SoundCloud Public API Specification](https://developers.soundcloud.com/docs/api/explorer/open-api#/)

Nạp các thư viện cần thiết

In [1]:
import sys
!pip install --upgrade concurrent
import requests
import time
import json
import csv 
import pandas as pd
from concurrent import futures

ERROR: Could not find a version that satisfies the requirement concurrent
ERROR: No matching distribution found for concurrent


Cần gán đuôi params vì nếu không có `client_id`, ta sẽ không request được (trả về `status_code` là `4xx`)

In [2]:
params = (
    ('client_id', '6gsNBd4mJwXr0LxTBh8VKBOrViK6Aj56'),
)

Duyệt qua các ID của playlists, khoảng hơn 10000 links vì chỉ có khoảng 1/10 trong số đó request thành công. Nên để tránh việc các link trả về lỗi, ta request số lượng lớn link.

Từ playlists, ta lấy các ID của các tracks và users, để đảm bảo cả 3 entity dữ liệu có liên quan đến nhau.

## Lấy dữ liệu playlists

In [3]:
start = time.time()
search_base_link = 'https://api-v2.soundcloud.com/playlists/'
numbers = [i for i in range(1, 11000)]
api_links = [search_base_link + str(number) for number in numbers]
tracksID = []
usersID = []
count = 1
def get_playlists(url):
    global count
    response = requests.get(url, params=params)
    
    # kiểm tra status code
    if response.status_code==200:
        
        # encode format utf-8
        response.encoding = response.apparent_encoding
        
        # lấy dữ liệu dưới dạng json
        data = json.loads(response.text)
        
        # kiểm tra json có tồn tại hay không
        if data:
            if data['tracks']:
                # lấy dữ liệu các tracks nằm trong playlist
                tracksID.extend([id_['id'] for id_ in data['tracks']])
                
                # gán lại thuộc tính tracks (chỉ lấy id track), từ {[artist_: abc, id: 123...]} sang [123, ...] 
                data['tracks'] = [id_['id'] for id_ in data['tracks']]
            
            # lấy dữ liệu users sở hữu playlist
            usersID.append(data['user_id'])
                
            try:
                # nếu là mở file lần đầu tiên thì lưu thêm header
                if count == 1:
                    with open('./Api_data/playlists.csv', 'w', newline='') as file:
                        writer = csv.writer(file)
                        header = data.keys()
                        writer.writerow(header)
                        
                # lưu data vào file
                with open('./Api_data/playlists.csv', 'a', newline='') as file:
                    writer = csv.writer(file)
                    writer.writerow(data.values())
                count += 1
            except Exception as e:
                # thông thường sẽ có lỗi về format charcode lúc lưu file 
                # (vì những kí tự đặc biệt trong data), khi có lỗi sẽ in ra
                # số lượng playlist lỗi không đáng kể
                pass

with futures.ThreadPoolExecutor(max_workers=25) as executor:
    executor.map(get_playlists, api_links)

Thời gian chạy

In [4]:
print(time.time() - start)

233.59122252464294


tracks ID và users ID lấy được khi request data playlist

In [5]:
len(tracksID)

6437

In [6]:
len(usersID)

1469

Sau khi lưu vào file, đọc lên để kiểm tra

In [7]:
df = pd.read_csv('./Api_data/playlists.csv')
df

,artwork_url,created_at,description,duration,embeddable_by,genre,id,kind,label_name,last_modified,...,title,uri,user_id,set_type,is_album,published_at,display_date,user,tracks,track_count
0,com/releases/SK006CD,2003-06-02T00:00:00Z,262,NaN,public,downtempo sampling beats,Soulhack,https://api.soundcloud.com/playlists/18,183,album,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,2008-05-01T17:31:06Z,NaN,50620378.0,all,NaN,34,playlist,NaN,2021-10-16T14:55:49Z,...,Dj Mixes,https://api.soundcloud.com/playlists/34,2157.0,NaN,False,NaN,2008-05-01T17:31:06Z,{'avatar_url': 'https://i1.sndcdn.com/avatars-...,"[859159762, 438021198, 274210741, 274210163, 2...",10.0
2,NaN,2008-05-05T03:06:57Z,songs i made for a rave i played at in 2007,2498158.0,all,smoke & chill,38,playlist,NaN,2015-10-13T13:39:25Z,...,the rave songs,https://api.soundcloud.com/playlists/38,1719.0,ep,True,NaN,2008-05-05T03:06:57Z,{'avatar_url': 'https://i1.sndcdn.com/avatars-...,"[4433, 4432, 4431, 4430]",4.0
3,NaN,2008-05-21T21:29:48Z,\nThis time we bring you a complete rerelease ...,0.0,all,NaN,59,playlist,NaN,2008-05-21T21:29:48Z,...,EEVO001R US Heritage,https://api.soundcloud.com/playlists/59,2314.0,NaN,False,NaN,2008-05-21T21:29:48Z,{'avatar_url': 'https://i1.sndcdn.com/avatars-...,[],0.0
4,NaN,2008-05-14T04:19:50Z,mute1945 is a very very unique artist and geni...,489215.0,all,non genre,48,playlist,Floor Limit,2008-05-14T04:31:39Z,...,mute1945 - karte6524,https://api.soundcloud.com/playlists/48,1691.0,album,True,NaN,2008-05-14T04:19:50Z,{'avatar_url': 'https://i1.sndcdn.com/avatars-...,[],0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1444,https://i1.sndcdn.com/artworks-000003550697-6v...,2009-03-19T04:07:06Z,NaN,3596566.0,all,Dubstep,10973,playlist,NaN,2011-05-20T02:14:27Z,...,TOTAL RECALL MUSIC PLAYER,https://api.soundcloud.com/playlists/10973,52713.0,NaN,False,NaN,2009-03-19T04:07:06Z,{'avatar_url': 'https://i1.sndcdn.com/avatars-...,"[15590228, 12821206, 11326904, 11219640, 11219...",23.0
1445,https://i1.sndcdn.com/artworks-000000393201-9d...,2009-03-19T06:27:40Z,Mathon is a village in the swiss mountains on ...,2431485.0,all,Ambient,10980,playlist,NaN,2015-10-13T13:39:33Z,...,Mathon - Muntsulej,https://api.soundcloud.com/playlists/10980,27184.0,album,True,NaN,2009-03-19T06:27:40Z,{'avatar_url': 'https://i1.sndcdn.com/avatars-...,"[189418, 189420, 1524366, 1524830, 1525318]",5.0
1446,NaN,2009-03-19T07:44:15Z,Latest releases,10587147.0,me,Techno,10986,playlist,NaN,2015-04-01T11:21:13Z,...,Latest releases,https://api.soundcloud.com/playlists/10986,14408.0,NaN,False,NaN,2009-03-19T07:44:15Z,{'avatar_url': 'https://i1.sndcdn.com/avatars-...,"[192581309, 192582130, 192583101, 184139957, 1...",24.0
1447,https://i1.sndcdn.com/artworks-000000469010-z3...,2009-03-19T06:26:58Z,music by QUIX 05 and Elliot Caps,4314948.0,all,NaN,10979,playlist,NaN,2009-10-04T04:31:15Z,...,Phone Home Recs,https://api.soundcloud.com/playlists/10979,42854.0,NaN,False,NaN,2009-03-19T06:26:58Z,{'avatar_url': 'https://i1.sndcdn.com/avatars-...,"[391552, 159570, 150707, 153162398]",4.0


## Lấy dữ liệu tracks

Tương tự như playlists, tracks được lấy tương tự, nhưng track ID sẽ được request sẽ lấy từ mảng `tracksID` ở trên.

In [8]:
search_base_link = 'https://api-v2.soundcloud.com/tracks/'
api_links = [search_base_link + str(number) for number in tracksID]
not_done_tracks = []
count = 1
start = time.time()
def get_api_tracks_1(url):
    global count
    response = requests.get(url, params=params)
    if response.status_code==200:
        response.encoding = response.apparent_encoding
        data = json.loads(response.text)
        if data:
            # không cần thuộc tính user, vì đã có user_id
            # có thể truy cập từ file users.csv
            data = data.pop('user')
            try:
                if count == 1:
                    with open('./Api_data/tracks.csv', 'w', newline='') as file:
                        writer = csv.writer(file)
                        header = data.keys()
                        writer.writerow(header)
                with open('./Api_data/tracks.csv', 'a', newline='') as file:
                    writer = csv.writer(file)
                    writer.writerow(data.values())
                count += 1
            except Exception as e:
                pass
    else:
        # những link khi request thất bại sẽ được lấy để request lại
        not_done_tracks.append(url)
    
with futures.ThreadPoolExecutor(max_workers=25) as executor:
    executor.map(get_api_tracks_1, api_links)

Thời gian chạy

In [9]:
print(time.time() - start)

156.78331756591797


Số lượng link request thất bại

In [10]:
len(not_done_tracks)

3251

Request các link đã thất bại trong lần request đầu tiên

In [11]:
start = time.time()
def get_api_tracks_2(url):
    global count
    response = requests.get(url, params=params)
    if response.status_code==200:
        response.encoding = response.apparent_encoding
        data = json.loads(response.text)
        if data:
            data = data.pop('user')
            try:
                with open('./Api_data/tracks.csv', 'a', newline='') as file:
                    writer = csv.writer(file)
                    writer.writerow(data.values())
                count += 1
            except Exception as e:
                pass
    
with futures.ThreadPoolExecutor(max_workers=25) as executor:
    executor.map(get_api_tracks_2, not_done_tracks)

Thời gian chạy

In [12]:
print(time.time() - start)

79.00036382675171


Sau khi lấy dữ liệu, đọc file để kiểm tra lại lần nữa

In [13]:
df = pd.read_csv('./Api_data/tracks.csv')
df

,avatar_url,first_name,followers_count,full_name,id,kind,last_modified,last_name,permalink,permalink_url,uri,urn,username,verified,city,country_code,badges,station_urn,station_permalink
0,https://i1.sndcdn.com/avatars-000012778523-59q...,Eric,133692,Eric Wahlforss,183,user,2020-12-16T21:40:33Z,Wahlforss,forss,https://soundcloud.com/forss,https://api.soundcloud.com/users/183,soundcloud:users:183,Forss,True,Berlin,DE,"{'pro': False, 'pro_unlimited': False, 'verifi...",soundcloud:system-playlists:artist-stations:183,artist-stations:183
1,https://i1.sndcdn.com/avatars-N3HNyXVCJnJ8YiLQ...,Kim,1502,Kim,2157,user,2021-08-17T10:55:32Z,NaN,kimik,https://soundcloud.com/kimik,https://api.soundcloud.com/users/2157,soundcloud:users:2157,Kimik,False,Tampere,FI,"{'pro': False, 'pro_unlimited': True, 'verifie...",soundcloud:system-playlists:artist-stations:2157,artist-stations:2157
2,https://i1.sndcdn.com/avatars-N3HNyXVCJnJ8YiLQ...,Kim,1502,Kim,2157,user,2021-08-17T10:55:32Z,NaN,kimik,https://soundcloud.com/kimik,https://api.soundcloud.com/users/2157,soundcloud:users:2157,Kimik,False,Tampere,FI,"{'pro': False, 'pro_unlimited': True, 'verifie...",soundcloud:system-playlists:artist-stations:2157,artist-stations:2157
3,https://i1.sndcdn.com/avatars-N3HNyXVCJnJ8YiLQ...,Kim,1502,Kim,2157,user,2021-08-17T10:55:32Z,NaN,kimik,https://soundcloud.com/kimik,https://api.soundcloud.com/users/2157,soundcloud:users:2157,Kimik,False,Tampere,FI,"{'pro': False, 'pro_unlimited': True, 'verifie...",soundcloud:system-playlists:artist-stations:2157,artist-stations:2157
4,https://i1.sndcdn.com/avatars-N3HNyXVCJnJ8YiLQ...,Kim,1502,Kim,2157,user,2021-08-17T10:55:32Z,NaN,kimik,https://soundcloud.com/kimik,https://api.soundcloud.com/users/2157,soundcloud:users:2157,Kimik,False,Tampere,FI,"{'pro': False, 'pro_unlimited': True, 'verifie...",soundcloud:system-playlists:artist-stations:2157,artist-stations:2157
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4761,https://i1.sndcdn.com/avatars-ZuGaiU2ByzL3ejva...,Fredrick,1434,Fredrick Berntsson,14408,user,2021-08-05T06:43:02Z,Berntsson,fred-white,https://soundcloud.com/fred-white,https://api.soundcloud.com/users/14408,soundcloud:users:14408,Fred White,False,Arvika,SE,"{'pro': False, 'pro_unlimited': True, 'verifie...",soundcloud:system-playlists:artist-stations:14408,artist-stations:14408
4762,https://i1.sndcdn.com/avatars-ZuGaiU2ByzL3ejva...,Fredrick,1434,Fredrick Berntsson,14408,user,2021-08-05T06:43:02Z,Berntsson,fred-white,https://soundcloud.com/fred-white,https://api.soundcloud.com/users/14408,soundcloud:users:14408,Fred White,False,Arvika,SE,"{'pro': False, 'pro_unlimited': True, 'verifie...",soundcloud:system-playlists:artist-stations:14408,artist-stations:14408
4763,https://i1.sndcdn.com/avatars-ZuGaiU2ByzL3ejva...,Fredrick,1434,Fredrick Berntsson,14408,user,2021-08-05T06:43:02Z,Berntsson,fred-white,https://soundcloud.com/fred-white,https://api.soundcloud.com/users/14408,soundcloud:users:14408,Fred White,False,Arvika,SE,"{'pro': False, 'pro_unlimited': True, 'verifie...",soundcloud:system-playlists:artist-stations:14408,artist-stations:14408
4764,https://i1.sndcdn.com/avatars-000365084324-iuy...,Trance,7489,Trance Poetry,57,user,2019-02-27T13:46:38Z,Poetry,trancepoetry,https://soundcloud.com/trancepoetry,https://api.soundcloud.com/users/57,soundcloud:users:57,@trancepoetry,False,The Internet,SE,"{'pro': False, 'pro_unlimited': True, 'verifie...",soundcloud:system-playlists:artist-stations:57,artist-stations:57


## Lấy dữ liệu users

Tương tự như playlists, users được lấy tương tự, nhưng user ID sẽ được request sẽ lấy từ mảng `usersID` ở trên.

In [14]:
search_base_link = 'https://api-v2.soundcloud.com/users/'
api_links = [search_base_link + str(number) for number in usersID]
not_done_users = []
count = 1
start = time.time()
def get_api_users(url):
    global count
    response = requests.get(url, params=params)
    if response.status_code==200:
        response.encoding = response.apparent_encoding
        data = json.loads(response.text)
        if data:
            try:
                if count == 1:
                    with open('./Api_data/users.csv', 'w', newline='') as file:
                        writer = csv.writer(file)
                        header = data.keys()
                        writer.writerow(header)
                with open('./Api_data/users.csv', 'a', newline='') as file:
                    writer = csv.writer(file)
                    writer.writerow(data.values())
                count += 1
            except Exception as e:
                pass
    else:
        not_done_users.append(url)
    
with futures.ThreadPoolExecutor(max_workers=25) as executor:
    executor.map(get_api_users, api_links)

Thời gian chạy

In [15]:
print(time.time() - start)

30.09919548034668


Số lượng link request thất bại

In [16]:
len(not_done_users)

751

Request các link đã thất bại trong lần request đầu tiên

In [17]:
start = time.time()
def get_api_users(url):
    global count
    response = requests.get(url, params=params)
    if response.status_code==200:
        response.encoding = response.apparent_encoding
        data = json.loads(response.text)
        if data:
            try:
                with open('./Api_data/users.csv', 'a', newline='') as file:
                    writer = csv.writer(file)
                    writer.writerow(data.values())
                count += 1
            except Exception as e:
                pass
    
with futures.ThreadPoolExecutor(max_workers=25) as executor:
    executor.map(get_api_users, not_done_users)

Thời gian chạy

In [18]:
print(time.time() - start)

15.649960279464722


Sau khi lấy dữ liệu, đọc file lên để kiểm tra

In [19]:
df = pd.read_csv('./Api_data/users.csv')
df

,avatar_url,city,comments_count,country_code,created_at,creator_subscriptions,creator_subscription,description,followers_count,followings_count,...,reposts_count,track_count,uri,urn,username,verified,visuals,badges,station_urn,station_permalink
0,https://i1.sndcdn.com/avatars-000001600946-mwn...,Stockholm,30,SE,2007-09-27T00:17:00Z,[{'product': {'id': 'free'}}],{'product': {'id': 'free'}},Bruce Brus is Canadian/Swedish musician & prod...,4032,677,...,NaN,15,https://api.soundcloud.com/users/231,soundcloud:users:231,Owen Meyers / Bruce Brus,False,"{'urn': 'soundcloud:users:231', 'enabled': Tru...","{'pro': False, 'pro_unlimited': False, 'verifi...",soundcloud:system-playlists:artist-stations:231,artist-stations:231
1,https://i1.sndcdn.com/avatars-N3HNyXVCJnJ8YiLQ...,Tampere,241,FI,2008-04-19T17:43:00Z,[{'product': {'id': 'creator-pro-unlimited'}}],{'product': {'id': 'creator-pro-unlimited'}},I <3 synths. 1/2 of U&K. Host of Etkoplasma ra...,1502,2002,...,NaN,73,https://api.soundcloud.com/users/2157,soundcloud:users:2157,Kimik,False,"{'urn': 'soundcloud:users:2157', 'enabled': Tr...","{'pro': False, 'pro_unlimited': True, 'verifie...",soundcloud:system-playlists:artist-stations:2157,artist-stations:2157
2,https://i1.sndcdn.com/avatars-000365084324-iuy...,The Internet,80,SE,2007-08-22T16:40:00Z,[{'product': {'id': 'creator-pro-unlimited'}}],{'product': {'id': 'creator-pro-unlimited'}},NaN,7489,643,...,NaN,20,https://api.soundcloud.com/users/57,soundcloud:users:57,@trancepoetry,False,"{'urn': 'soundcloud:users:57', 'enabled': True...","{'pro': False, 'pro_unlimited': True, 'verifie...",soundcloud:system-playlists:artist-stations:57,artist-stations:57
3,https://i1.sndcdn.com/avatars-000000018772-c5d...,Tokyo,0,JP,2008-03-06T12:56:42Z,[{'product': {'id': 'free'}}],{'product': {'id': 'free'}},Floor Limit is the music label in Japan.We hav...,265,5,...,NaN,5,https://api.soundcloud.com/users/1691,soundcloud:users:1691,ark,False,NaN,"{'pro': False, 'pro_unlimited': False, 'verifi...",soundcloud:system-playlists:artist-stations:1691,artist-stations:1691
4,https://i1.sndcdn.com/avatars-000183177794-ps2...,Borlänge,9,SE,2008-05-14T19:44:36Z,[{'product': {'id': 'free'}}],{'product': {'id': 'free'}},NaN,209,46,...,NaN,29,https://api.soundcloud.com/users/2455,soundcloud:users:2455,deeped,False,"{'urn': 'soundcloud:users:2455', 'enabled': Tr...","{'pro': False, 'pro_unlimited': False, 'verifi...",soundcloud:system-playlists:artist-stations:2455,artist-stations:2455
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1056,https://i1.sndcdn.com/avatars-000000101972-a6e...,London,113,GB,2008-10-14T23:05:23Z,[{'product': {'id': 'creator-pro-unlimited'}}],{'product': {'id': 'creator-pro-unlimited'}},NaN,184,295,...,NaN,35,https://api.soundcloud.com/users/23025,soundcloud:users:23025,QuantumLoop,False,NaN,"{'pro': False, 'pro_unlimited': True, 'verifie...",soundcloud:system-playlists:artist-stations:23025,artist-stations:23025
1057,https://a1.sndcdn.com/images/default_avatar_la...,Brooklyn,0,US,2009-03-17T03:12:36Z,[{'product': {'id': 'free'}}],{'product': {'id': 'free'}},Throcke is the creation of Christopher Molline...,6,0,...,NaN,5,https://api.soundcloud.com/users/86395,soundcloud:users:86395,throcke,False,NaN,"{'pro': False, 'pro_unlimited': False, 'verifi...",soundcloud:system-playlists:artist-stations:86395,artist-stations:86395
1058,https://a1.sndcdn.com/images/default_avatar_la...,London,0,GB,2009-03-17T23:56:27Z,[{'product': {'id': 'free'}}],{'product': {'id': 'free'}},NaN,2,1,...,NaN,0,https://api.soundcloud.com/users/86829,soundcloud:users:86829,detoxmusic1,False,NaN,"{'pro': False, 'pro_unlimited': False, 'verifi...",soundcloud:system-playlists:artist-stations:86829,artist-stations:86829
1059,https://i1.sndcdn.com/avatars-000023654001-49d...,Athens,0,GR,2009-03-18T09:16:38Z,[{'product': {'id': 'free'}}],{'product': {'id': 'free'}},NaN,14,0,...,NaN,6,https://api.soundcloud.com/users/86946,soundcloud:user